In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
import os
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.environ["OPENAI_PRIVATE_API_KEY"])

In [4]:
class AgentState(TypedDict):
    task: str
    plan: str
    plan_critique: str
    plan_revision_number: int
    plan_max_revisions: int

    sections: List[str]

    research_critique: List[str]
    research_revision_number: int
    research_max_revisions: int

    content: List[str]

    draft: List[str]
    draft_critique: List[str]
    draft_revision_number: int
    draft_max_revisions: int

    css: str
    html: str


In [5]:
PLAN_PROMPT = """Jsi expertem na analýzu a popis společností, firem či organizací a máš za úkol napsat \
    vysokoúrovňový plán odborného reportu popisujícího zadanou společnost. Uveď plán reportu spolu \
    s příslušnými poznámkami a pokyny pro jednotlivé sekce a podsekce. Tak, aby uživatel tohoto reportu \
    obdržel celkový obraz o firmě. \
    
    Pokud uživatel poskytne kritiku plánu, odpověz upravenou verzí plánu, která zohlední jeho připomínky. \
    Připomínky: \

    {plan_critique}
    """

In [6]:
PLAN_CRITIQUE_PROMPT = """Jsi manažerem společnosti, zkontroluj, že plán reportu obsahuje všechny potřebné \
    informace o firmě a je dostatečně srozumitelný. Pokud něco chybí nebo je nesrozumitelné, uveď chybějící \
    nebo nesrozumitelné části a navrhni, jak by se měly upravit. \
    
    Plán reportu: \
    {plan}"""

In [7]:
RESEARCH_PLAN_PROMPT_OBSOLETE = """Jsi specialista na informační průzkum. Tvým úkolem je efektivně prohledávat \
    internet a tvořit dotazy, které pokryjí co největší množství relevantních informací k plánu reportu o firmě {task}. \
    S následujícím plánem reportu: \
    {plan} \
    Vygeneruj seznam vyhledávacích dotazů, které shromáždí relevantní informace. \
    Vygeneruj nejvýše 5 dotazů.\
    
    Pokud uživatel poskytne kritiku na nedostatek informací, odpověz upravenou verzí plánu, která zohlední jeho připomínky. \
    
    Připomínky: \
    {research_critique}
    """

In [8]:
RESEARCH_PLAN_PROMPT = """Jsi specialista na informační průzkum. Tvým úkolem je efektivně prohledávat \
    internet a tvořit dotazy, které pokryjí co největší množství relevantních informací k zadané sekci plánu reportu o firmě {task}. \
    
    Plán reportu: \
    {plan} \

    Zpracovávaná sekce reportu: \
    {section} \
    
    Vygeneruj seznam vyhledávacích dotazů, které shromáždí relevantní informace. \
    Vygeneruj nejvýše 3 dotazy.\
    
    Pokud uživatel poskytne kritiku na nedostatek informací, odpověz upravenými dotazy, která zohlední jeho připomínky. \
    
    Připomínky: \
    {research_critique}
    """

In [9]:
RESEARCH_CRITIQUE_PROMPT = """Jsi manažerem společnosti, zkontroluj, že pro zadaný plán reportu a konkrétní zpracovávanou sekci, \
    byly nalezeny všechny potřebné informace. Pokud něco chybí, pak uveď chybějící části a navrhni informace, \
    které by se měly vyhledat. \
    
    Plán reportu: \
    {plan}

    Zpracovávaná sekce reportu: \
    {section}
    
    Vyhledané informace: \
    {research}
    """

In [10]:
WRITER_PROMPT = """Jsi expertem na psaní odborných textů. Tvým úkolem je v několika odstavcích napsat sekci odborného reportu o firmě {task}. \
    Sekce reportu by měla obsahovat všechny informace z plánu reportu a informace získané z internetu. \
    Sekce reportu by měla být srozumitelná a obsahovat všechny potřebné informace. \
    Každá sekce reportu by měla končit seznam odkazů, ze kterých bylo čerpáno. \
    
    Pokud uživatel poskytne kritiku na nedostatek informací, odpověz upravenou verzí reportu, která zohlední jeho připomínky. \
    
    Připomínky: \
    {draft_critique}

    Plán reportu: \
    {plan}

    Zpraovávaná sekce reportu: \
    {section}

    Vyhledané informace: \
    {research}
    """

In [11]:
WRITER_CRITIQUE_PROMPT = """Jsi manažerem společnosti, zkontroluj, že report obsahuje všechny potřebné informace o firmě {task} \
    a je dostatečně srozumitelný. Pokud něco chybí nebo je nesrozumitelné, uveď chybějící nebo nesrozumitelné části \
    a navrhni, jak by se měly upravit. \
    
    Report: \
    {draft}

    Plán reportu: \
    {plan}
    """

In [12]:
CSS_PROMPT = """Jsi expertem na tvorbu webových stránek a kaskádových stylů a máš za úkol vytvořit CSS styl pro webovou stránku, \
    která bude zobrazovat report o firmě {task} na základě plánu reportu. Uveď CSS styl, který bude použit pro webovou stránku. \

    Plán reportu: \
    {plan}
"""

In [13]:
HTML_SECTION_GENERATOR_PROMPT = """Jsi expertem na tvorbu webových stránek, který má za úkol vytvořit HTML kód pro jednu konkrétní sekci plánu reportu pro popis firmy {task}. \
    Vygeneruj HTML kód vybrané sekce včetně odkazů na zdroje, kde čeština bude kódována v utf8, který zobrazí text sekce prohlížeči. \
    Tvým úkolem není vytvořit celý HTML dokument, ale pouze jeden kontejner obsahující konkrétní sekci. \
    Jako styl použij CSS styl uvedený níže, který byl vytvořen pro tento článek.\

    Text sekce: \
    {draft}

    CSS styl: \
    {css}
"""

In [14]:
HTML_GENERATOR_PROMPT = """Jsi expertem na tvorbu webových stránek, který má za úkol vytvořit HTML verzi popisu firmy {task}. \
    Vygeneruj HTML kód, kde čeština bude kódována v utf8, jehož cílem bude zobrazit popips firmy na základě plánu reportu. \
    Na základě plánu reportu mi vygeneruj v HTML speciální značku pro komentář <!--section_id--> , kde id bude číselné označení sekce. \    
    Tuto speciální značku následně použiji pro nahrazení konkrétním obsahem sekce. \
    Do těla HTML doplň pouze pod sebe tyto speciální značky a nevkládej žádný další obsah, tagy ani nic jiného. \
    Jako styl použij CSS styl, který byl vytvořen pro tento článek. Daný styl zahrň jako součást webové stránky.\
    
    Speciální značka: \
    <!--section_id-->

    Plán reportu: \
    {plan}

    CSS styl: \
    {css}
"""

In [15]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

class Sections(BaseModel):
    sections: List[str]

In [16]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [17]:
def plan_node_obsolete(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT.format(plan_critique=state["plan_critique"])), 
        HumanMessage(content="Vytvoř plán reportu popisujícího společnost."),
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT.format(plan_critique=state["plan_critique"])), 
        HumanMessage(content="Vytvoř plán reportu popisujícího společnost."),
    ]
    response = model.with_structured_output(Sections).invoke(messages)

    content = ""
    for section in response.sections:
        content += f"## {section}\n\n"        

    return {"plan": content, "sections": response.sections}

In [18]:
def plan_critique_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_CRITIQUE_PROMPT.format(plan=state["plan"])), 
        HumanMessage(content="Vytvoř kritiku plánu reportu.")
    ]
    response = model.invoke(messages)    
    return {
        "plan_critique": response.content, 
        "plan_revision_number": state.get("plan_revision_number", 1) + 1
    }

In [19]:
def research_plan_node_obsolete(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT.format(research_critique=state["research_critique"], task=state["task"], plan=state["plan"])),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=5)        
        for r in response['results']:
            #content.append(r['content'])
            content.append(f"{r['title']}\n{r['content']}\n{r['url']}\n")
    return {"content": content}

In [20]:
def research_plan_node(state: AgentState):
    contents = []
    for section_id, section in enumerate(state["sections"]):
        research_critique = state["research_critique"][section_id] if state["research_critique"] != None and section_id < len(state["research_critique"]) else ""
        queries = model.with_structured_output(Queries).invoke([
            SystemMessage(content=RESEARCH_PLAN_PROMPT.format(research_critique=research_critique, task=state["task"], section=section, plan=state["plan"])),
            HumanMessage("Vytvoř seznam vyhledávacích dotazů pro danou sekci plánu reportu.")
        ])
        content = [] if state['content'] == None else state['content'][section_id] or []
        #content = state['content'][section_id] or []
        for q in queries.queries:
            response = tavily.search(query=q, max_results=3)        
            for r in response['results']:                
                content.append(f"{r['title']}\n{r['content']}\n{r['url']}\n")
        contents.append(content)
        
    return {"content": contents}

In [21]:
def research_critique_node_obsolete(state: AgentState):    
    messages = [
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT.format(plan=state["plan"], research=state["content"])), 
        HumanMessage(content="Zhodnoť poskytnutý obsah pro zadaný plán reportu.")
    ]
    response = model.invoke(messages)    
    return {
        "research_critique": response.content, 
        "research_revision_number": state.get("research_revision_number", 1) + 1
    }

In [22]:
def research_critique_node(state: AgentState):    
    critiques = []
    for section_id, section in enumerate(state["sections"]):
        messages = [
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT.format(plan=state["plan"], research=state["content"][section_id], section=section)), 
            HumanMessage(content="Zhodnoť poskytnutý obsah pro zadaný plán reportu a zpracovávanou sekci.")
        ]
        response = model.invoke(messages)

        critiques.append(response.content)
        
    return {
        "research_critique": critiques,
        "research_revision_number": state.get("research_revision_number", 1) + 1
    }

In [23]:
def writer_node_obsolete(state: AgentState):
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(draft_critique=state["draft_critique"], task=state["task"], plan=state["plan"], research=state["content"])), 
        HumanMessage(content="Napiš odborný report o firmě.")
    ]
    response = model.invoke(messages)
    return {"draft": response.content}

In [24]:
def writer_node(state: AgentState):    
    drafts = []
    for section_id, section in enumerate(state["sections"]):
        draft_critique = state["draft_critique"][section_id] if state["draft_critique"] != None and section_id < len(state["draft_critique"]) else ""            
        messages = [
            SystemMessage(content=WRITER_PROMPT.format(draft_critique=draft_critique, task=state["task"], plan=state["plan"], research=state["content"][section_id], section=section)), 
            HumanMessage(content="Napiš odborný report o firmě.")
        ]
        response = model.invoke(messages)
        drafts.append(response.content)
        
    return {"draft": drafts}

In [25]:
def writer_critique_node(state: AgentState):
    critiques = []
    for draft_id, draft in enumerate(state["draft"]):
        messages = [
            SystemMessage(content=WRITER_CRITIQUE_PROMPT.format(draft=draft, plan=state["plan"], task=state["task"], section=state["sections"][draft_id])), 
            HumanMessage(content="Zhodnoť poskytnutý obsah pro zadaný plán reportu a právě zpracovávanou sekci.")
        ]
        response = model.invoke(messages)
        critiques.append(response.content)

    return {
        "draft_critique": critiques,
        "draft_revision_number": state.get("draft_revision_number", 1) + 1
    }

In [26]:
def css_node(state: AgentState):
    messages = [
        SystemMessage(content=CSS_PROMPT.format(plan=state["plan"], task=state["task"])), 
        HumanMessage(content="Vytvoř CSS styl pro webovou stránku.")
    ]
    response = model.invoke(messages)
    return {"css": response.content}

In [27]:
def html_generator_node(state: AgentState):
    messages = [
        SystemMessage(content=HTML_GENERATOR_PROMPT.format(plan=state["plan"], css=state["css"], task=state["task"])), 
        HumanMessage(content="Vygeneruj HTML kód pro webovou stránku s označneými místy pro sekce.")
    ]
    empty_html = model.invoke(messages).content

    store_txt(empty_html, "empy.html")
    
    for draft_id, draft in enumerate(state["draft"]):
        messages = [
            SystemMessage(content=HTML_SECTION_GENERATOR_PROMPT.format(draft=draft, css=state["css"], task=state["task"])), 
            HumanMessage(content="Vygeneruj HTML kód pro konkrétní sekci.")
        ]
        response = model.invoke(messages)
        store_txt(response.content, str(draft_id)+".html")
        empty_html = empty_html.replace("<!--section_" + str(draft_id+1) + "-->", f"{extract_html(response.content)}\n\n")
        
    return {"html": empty_html}

In [28]:
def debug_node(state: AgentState):
    f = open("draft.md", "w", encoding="utf-8")
    f.write(state["draft"])
    f.close()
    return {"task": "hotovo"}

In [29]:
def should_continue(state):    
    if state["plan_revision_number"] >= state["plan_max_revisions"]:
        return "done"
    return "reflect"

In [30]:
def should_continue_research(state):
    if state["research_revision_number"] >= state["research_max_revisions"]:
        return "done"
    return "research"

In [31]:
def should_continue_writer(state):
    if state["draft_revision_number"] >= state["draft_max_revisions"]:
        return "done"
    return "write"

In [32]:
import re

def extract_html(html: str):
    pattern = r'```html(.*?)```'
    match = re.search(pattern, html, re.DOTALL)
    return match.group(1)

In [33]:
def store_txt(txt: str, filename: str):
    f = open(filename, "w", encoding="utf-8")
    f.write(txt)
    f.close()

In [34]:
import re

def store_html(state):
    f = open("index.html", "w", encoding="utf-8")    
    f.write(extract_html(state["html"]))
    f.close()
    return {"task": "hotovo"}

In [35]:
builder = StateGraph(AgentState)
builder.add_node("planner", plan_node)
builder.add_node("plan_reflect", plan_critique_node)

builder.add_node("generate_queries", research_plan_node)
builder.add_node("research_reflect", research_critique_node)

builder.add_node("write", writer_node)
builder.add_node("write_reflect", writer_critique_node)

builder.add_node("css_node", css_node)
builder.add_node("html_node", html_generator_node)
builder.add_node("store", store_html)

In [36]:
builder.add_conditional_edges("planner", should_continue, {
    "done": "generate_queries",
    "reflect": "plan_reflect"    
})

builder.add_edge("plan_reflect", "planner")
builder.add_conditional_edges("generate_queries", should_continue_research, {
    "done": "write",
    "research": "research_reflect"    
})

builder.add_edge("research_reflect", "generate_queries")

builder.add_conditional_edges("write", should_continue_writer, {
    "done": "css_node",
    "write": "write_reflect"
})

builder.add_edge("write_reflect", "write")
builder.add_edge("css_node", "html_node")
builder.add_edge("html_node", "store")

builder.add_edge("store", END)

In [37]:
builder.set_entry_point("planner")
graph = builder.compile(checkpointer=memory)

In [38]:
import time

thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': "VŠB - Technická univerzita Ostrava",
    "plan_max_revisions": 0,
    "plan_revision_number": 0,
    "research_max_revisions": 0,
    "research_revision_number": 0,
    "draft_max_revisions": 0,
    "draft_revision_number": 0,
}, thread):
    print(s, s.keys())    
    time.sleep(10)
    

{'planner': {'plan': '## 1. Úvod\n\n## 2. Historie společnosti\n\n## 3. Organizační struktura\n\n## 4. Produkty a služby\n\n## 5. Trh a konkurence\n\n## 6. Finanční analýza\n\n## 7. Strategie a vize\n\n## 8. Firemní kultura a hodnoty\n\n## 9. Závěr\n\n', 'sections': ['1. Úvod', '2. Historie společnosti', '3. Organizační struktura', '4. Produkty a služby', '5. Trh a konkurence', '6. Finanční analýza', '7. Strategie a vize', '8. Firemní kultura a hodnoty', '9. Závěr']}} dict_keys(['planner'])
{'generate_queries': {'content': [['VSB - Technical University of Ostrava - Wikipedia\nVSB - Technical University of Ostrava (abbreviated VSB-TUO; Czech: Vysoká škola báňská - Technická univerzita Ostrava) is a university in Ostrava in the Moravian-Silesian Region of the Czech Republic.It is a polytechnic university with a long history of high-quality engineering education and research. [2] The first Fraunhofer Innovation Platform for Applied Artificial Intelligence ...\nhttps://en.wikipedia.org/wik